### Using the metagenome calculated number of cells per m; for each species and the number of RSG cells calculate a weighted avg O2 consumed 
### Inputs:
/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GoM_Metagenome_analysis/GoM_metagenome_calculated_GEN_cells_per_ml.csv
/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv

/mnt/scgc/simon/microg2p/analyses/GORG_recruitment/RNA_O2_analysis/Average_O2_consumption_for_GENs_Greater_than_1perc_RSG.csv
/mnt/scgc/simon/microg2p/analyses/GORG_recruitment/210121_summary_tables/All_GoM_RNA_seq_transcripts_per_cell_by_gen_and_function_stacked.csv


### Outputs:

O2_normalization_cells_per_ml_calculations_20200202.csv
weighted_avg_O2_consumption_calculations.csv
GoM_mg_abundance_weighted_average_O2_consumption_per_cell.csv
/mnt/scgc/simon/microg2p/analyses/GoM_Summary_files/GoM_mg_abundance_weighted_average_O2_consumption_per_cell.csv
All_GoM_RNA_seq_transcripts_per_cell_by_gen_and_function_stacked_weighted_avg_O2_consumed_ALL_genes_all_GENs.csv


rerun on 2021-12-07 after correcting for salinity of media samples

In [23]:
outdir='/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/O2_consumption/'

In [24]:


import pandas as pd
from pandas import DataFrame
import os
import sys
import csv
from pathlib import Path
import seaborn as sns
sns.set()
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.show()
from collections import defaultdict
import os.path as op
import glob

os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/O2_consumption')
matplotlib.__version__
pd.set_option("max_rows", 10)

In [25]:
columns=['genus', '20171102_genome_size_normalized_metagenome_cells/ml','20181030_genome_size_normalized_metagenome_cells/ml',
         '20190402_genome_size_normalized_metagenome_cells/ml','20190709_genome_size_normalized_metagenome_cells/ml']

mg_abundance=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GoM_Metagenome_analysis/GoM_metagenome_calculated_Genus_cells_per_ml.csv', usecols=columns)
#mg_abundance=mg_abundance.drop(['20171102_mg_normalized_abundance', '20181030_mg_normalized_abundance', 
 #                              '20190402_mg_normalized_abundance', '20190709_mg_normalized_abundance', 'GEN', 'GEN_size_rank',
 #                              '# of cells', 'Best Classification'], axis=columns)
#mg_abundance=mg_abundance[mg_abundance.genus !='Unclassified']
mg_abundance['genus']=mg_abundance['genus'].astype(str)
mg_abundance

,genus,20171102_genome_size_normalized_metagenome_cells/ml,20181030_genome_size_normalized_metagenome_cells/ml,20190402_genome_size_normalized_metagenome_cells/ml,20190709_genome_size_normalized_metagenome_cells/ml
0,AAA164-E04,842.193966,1916.455808,369.342772,857.676906
1,AAA536-G10,17472.787073,16915.541760,14403.280557,17721.566151
2,AG-337-I02,2227.335509,9857.221878,1075.339238,1002.756455
3,AG-414-E02,125.191537,438.765267,401.157950,128.286002
4,AG-422-B15,1880.679178,3313.158567,13707.625915,6736.371699
...,...,...,...,...,...
163,Verrucomicrobiales,265.899026,397.482106,54.179686,280.890811
164,Vibrio,622.265686,620.661123,533.774653,964.925624
165,Vicingaceae,782.480532,470.155212,7151.270318,5084.185705
166,Winogradskyella,4635.264065,2001.192608,10605.994111,10722.976925


In [26]:
#Import SAG data csv file and parse it down so that it only includes the FRESH RSG SAGs

columns=['name', 'Sample_collection_Date', 'gate', 'GTDB_classification', 'FACS_date']

RSG_GENs=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv', 
                     usecols=columns)

RSG_GENs=RSG_GENs[RSG_GENs.gate == 'Respiring cells (RSG-positive)']
RSG_GENs['count']=1
RSG_GENs=RSG_GENs[RSG_GENs.Sample_collection_Date==RSG_GENs.FACS_date]
RSG_GENs=RSG_GENs.rename(columns={'GTDB_classification':'genus'})
RSG_GENs

,name,Sample_collection_Date,FACS_date,gate,genus,count
0,AH-135-A01,170412,170412,Respiring cells (RSG-positive),Pelagibacter,1
1,AH-135-A02,170412,170412,Respiring cells (RSG-positive),Hel1-33-131,1
2,AH-135-A03,170412,170412,Respiring cells (RSG-positive),IMCC9063,1
3,AH-135-A04,170412,170412,Respiring cells (RSG-positive),Thioglobus,1
4,AH-135-A05,170412,170412,Respiring cells (RSG-positive),SCGC-AAA076-P13,1
...,...,...,...,...,...,...
4728,AH-700-P10,190709,190709,Respiring cells (RSG-positive),Planktomarina,1
4729,AH-700-P11,190709,190709,Respiring cells (RSG-positive),Luminiphilus,1
4730,AH-700-P13,190709,190709,Respiring cells (RSG-positive),Planktomarina,1
4731,AH-700-P14,190709,190709,Respiring cells (RSG-positive),UBA8649,1


In [27]:
#create a pivot table of the counts of each gen per day that are labeled with RSG

RSG_pivot=pd.pivot_table(RSG_GENs, index='genus', values='count', columns='Sample_collection_Date', aggfunc=np.sum)
RSG_pivot=RSG_pivot.dropna(how='all', axis=0)
RSG_pivot



Sample_collection_Date,170412,170818,171102,181030,190402,190709
genus,,,,,,
AAA536-G10,1.0,4.0,2.0,3.0,5.0,1.0
AG-337-I02,NaN,1.0,1.0,4.0,NaN,NaN
AG-414-E02,1.0,NaN,NaN,NaN,NaN,NaN
AG-422-B15,1.0,2.0,1.0,NaN,3.0,NaN
ASP10-02a,5.0,NaN,NaN,NaN,4.0,NaN
...,...,...,...,...,...,...
UBA9926,3.0,NaN,4.0,NaN,2.0,4.0
Ulvibacter,2.0,NaN,NaN,NaN,1.0,NaN
Unclassified,76.0,77.0,97.0,44.0,55.0,60.0


In [28]:
#Sum the number of cells per day

RSG_cells_170412=RSG_pivot[170412].sum()
RSG_cells_170818=RSG_pivot[170818].sum()
RSG_cells_171102=RSG_pivot[171102].sum()
RSG_cells_181030=RSG_pivot[181030].sum()
RSG_cells_190402=RSG_pivot[190402].sum()
RSG_cells_190709=RSG_pivot[190709].sum()

In [29]:
#convert gens w/ <3 cells to Nan.
RSG_pivot.mask(RSG_pivot < 3, np.nan, inplace=True)
RSG_pivot=RSG_pivot.dropna(how='all', axis=0)
RSG_pivot

#and calculate the abundance of
RSG_pivot['170412_abundance'] = RSG_pivot[170412] / RSG_cells_170412
RSG_pivot['170818_abundance'] = RSG_pivot[170818] / RSG_cells_170818
RSG_pivot['171102_abundance'] = RSG_pivot[171102] / RSG_cells_171102
RSG_pivot['181030_abundance'] = RSG_pivot[181030] / RSG_cells_181030
RSG_pivot['190402_abundance'] = RSG_pivot[190402] / RSG_cells_190402
RSG_pivot['190709_abundance'] = RSG_pivot[190709] / RSG_cells_190709


RSG_pivot

/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

Sample_collection_Date,170412,170818,171102,181030,190402,190709,170412_abundance,170818_abundance,171102_abundance,181030_abundance,190402_abundance,190709_abundance
genus,,,,,,,,,,,,
AAA536-G10,NaN,4.0,NaN,3.0,5.0,NaN,NaN,0.013468,NaN,0.010638,0.018248,NaN
AG-337-I02,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,0.014184,NaN,NaN
AG-422-B15,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,0.010949,NaN
ASP10-02a,5.0,NaN,NaN,NaN,4.0,NaN,0.017668,NaN,NaN,NaN,0.014599,NaN
Amylibacter,15.0,5.0,8.0,4.0,15.0,5.0,0.053004,0.016835,0.026936,0.014184,0.054745,0.018051
...,...,...,...,...,...,...,...,...,...,...,...,...
UBA6772,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,0.014440
UBA7428,5.0,NaN,NaN,4.0,7.0,3.0,0.017668,NaN,NaN,0.014184,0.025547,0.010830
UBA9320,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,0.010830


In [30]:
#RSG cells/ml comes from counts of RSG labeled cells that are on my computer
#/Users/jmunson-mcgee/Desktop/ANI_clustering/2008_GEN_recreation/RSG_analysis/RSG_per_GEN_per_ml/RSG_cells_per_ml.csv
RSG_170412=8.13E+05
RSG_170818=7.20E+05
RSG_171102=6.45E+05
RSG_181030=6.42E+05
RSG_190402=6.80E+05
RSG_190709=1.00E+06

#calculate the number of cells/ml that are labeled with RSG for each gen/date
RSG_pivot['RSG_cells/ml_170412']=RSG_pivot['170412_abundance']*RSG_170412
RSG_pivot['RSG_cells/ml_170818']=RSG_pivot['170818_abundance']*RSG_170818
RSG_pivot['RSG_cells/ml_171102']=RSG_pivot['171102_abundance']*RSG_171102
RSG_pivot['RSG_cells/ml_181030']=RSG_pivot['181030_abundance']*RSG_181030
RSG_pivot['RSG_cells/ml_190402']=RSG_pivot['190402_abundance']*RSG_190402
RSG_pivot['RSG_cells/ml_190709']=RSG_pivot['190709_abundance']*RSG_190709

RSG_pivot

/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/mnt/scgc/scgc_nfs/opt/common/anaconda3a/envs/jacob_env/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithC

Sample_collection_Date,170412,170818,171102,181030,190402,190709,170412_abundance,170818_abundance,171102_abundance,181030_abundance,190402_abundance,190709_abundance,RSG_cells/ml_170412,RSG_cells/ml_170818,RSG_cells/ml_171102,RSG_cells/ml_181030,RSG_cells/ml_190402,RSG_cells/ml_190709
genus,,,,,,,,,,,,,,,,,,
AAA536-G10,NaN,4.0,NaN,3.0,5.0,NaN,NaN,0.013468,NaN,0.010638,0.018248,NaN,NaN,9696.969697,NaN,6829.787234,12408.759124,NaN
AG-337-I02,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,0.014184,NaN,NaN,NaN,NaN,NaN,9106.382979,NaN,NaN
AG-422-B15,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,0.010949,NaN,NaN,NaN,NaN,NaN,7445.255474,NaN
ASP10-02a,5.0,NaN,NaN,NaN,4.0,NaN,0.017668,NaN,NaN,NaN,0.014599,NaN,14363.957597,NaN,NaN,NaN,9927.007299,NaN
Amylibacter,15.0,5.0,8.0,4.0,15.0,5.0,0.053004,0.016835,0.026936,0.014184,0.054745,0.018051,43091.872792,12121.212121,17373.737374,9106.382979,37226.277372,18050.541516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UBA6772,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,0.014440,NaN,NaN,NaN,NaN,NaN,14440.433213
UBA7428,5.0,NaN,NaN,4.0,7.0,3.0,0.017668,NaN,NaN,0.014184,0.025547,0.010830,14363.957597,NaN,NaN,9106.382979,17372.262774,10830.324910
UBA9320,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,0.010830,NaN,NaN,NaN,NaN,NaN,10830.324910


In [31]:
#only keep the 4 timepoints with metagenome/metatranscriptome datasets and merge the dataframe with the mg cellular abundance
RSG_O2_GENs=RSG_pivot[RSG_pivot.columns[-4:]]
RSG_O2_GENs

Sample_collection_Date,RSG_cells/ml_171102,RSG_cells/ml_181030,RSG_cells/ml_190402,RSG_cells/ml_190709
genus,,,,
AAA536-G10,NaN,6829.787234,12408.759124,NaN
AG-337-I02,NaN,9106.382979,NaN,NaN
AG-422-B15,NaN,NaN,7445.255474,NaN
ASP10-02a,NaN,NaN,9927.007299,NaN
Amylibacter,17373.737374,9106.382979,37226.277372,18050.541516
...,...,...,...,...
UBA6772,NaN,NaN,NaN,14440.433213
UBA7428,NaN,9106.382979,17372.262774,10830.324910
UBA9320,NaN,NaN,NaN,10830.324910


In [32]:
O2_normalization=pd.merge(mg_abundance, RSG_O2_GENs, how='left', on='genus')
O2_normalization['genus']=O2_normalization['genus'].astype(str)
O2_normalization=O2_normalization.set_index('genus')
pd.set_option("max_rows", 170)
O2_normalization.head()

,20171102_genome_size_normalized_metagenome_cells/ml,20181030_genome_size_normalized_metagenome_cells/ml,20190402_genome_size_normalized_metagenome_cells/ml,20190709_genome_size_normalized_metagenome_cells/ml,RSG_cells/ml_171102,RSG_cells/ml_181030,RSG_cells/ml_190402,RSG_cells/ml_190709
genus,,,,,,,,
AAA164-E04,842.193966,1916.455808,369.342772,857.676906,NaN,NaN,NaN,NaN
AAA536-G10,17472.787073,16915.541760,14403.280557,17721.566151,NaN,6829.787234,12408.759124,NaN
AG-337-I02,2227.335509,9857.221878,1075.339238,1002.756455,NaN,9106.382979,NaN,NaN
AG-414-E02,125.191537,438.765267,401.157950,128.286002,NaN,NaN,NaN,NaN
AG-422-B15,1880.679178,3313.158567,13707.625915,6736.371699,NaN,NaN,7445.255474,NaN


In [33]:
# This is a little bit complicated but I am doing 2 things here for each date. I am calculating the number of cells above and below the O2 detection limit 
# for each GEN per day
# 1. In order to calculate the ones that are below the detection limit I check to make sure that the number of RSG cells/ml is < the number of mg adjusted cells
#    As long as it is I subtract the umber of RSG cells from the total number of cells. If it is not I enter 0

# 2. For cells above the detection limit it is a bit more complicated. first check to ensure that >0 were detected in the RSG sample if none were set this value = to 0
#    If more RSG cells were detected than mg normalized cells enter the mg normalized number of cells/ml
#.   If fewer RSG cells than mg normalized cells were detected enter the number of RSG cells/ml

O2_normalization=O2_normalization.fillna(0)

O2_normalization['20171102_cells_below_O2']= np.where (O2_normalization['RSG_cells/ml_171102']==0.000000, O2_normalization['20171102_genome_size_normalized_metagenome_cells/ml'],
                                                       (np.where(O2_normalization['RSG_cells/ml_171102']>=O2_normalization['20171102_genome_size_normalized_metagenome_cells/ml'], 0, 
                                                                 O2_normalization['20171102_genome_size_normalized_metagenome_cells/ml']-O2_normalization['RSG_cells/ml_171102'])))
O2_normalization['20171102_cells_above_O2']= np.where (O2_normalization['RSG_cells/ml_171102']==0,0,
                                                       (np.where(O2_normalization['RSG_cells/ml_171102']>=O2_normalization['20171102_genome_size_normalized_metagenome_cells/ml'], O2_normalization['20171102_genome_size_normalized_metagenome_cells/ml'], 
                                                                 O2_normalization['RSG_cells/ml_171102'])))

O2_normalization['20181030_cells_below_O2']= np.where (O2_normalization['RSG_cells/ml_181030']==0.000000, O2_normalization['20181030_genome_size_normalized_metagenome_cells/ml'],
                                                       (np.where(O2_normalization['RSG_cells/ml_181030']>=O2_normalization['20181030_genome_size_normalized_metagenome_cells/ml'], 0, 
                                                                 O2_normalization['20181030_genome_size_normalized_metagenome_cells/ml']-O2_normalization['RSG_cells/ml_181030'])))
O2_normalization['20181030_cells_above_O2']= np.where (O2_normalization['RSG_cells/ml_181030']==0,0,
                                                       (np.where(O2_normalization['RSG_cells/ml_181030']>=O2_normalization['20181030_genome_size_normalized_metagenome_cells/ml'], O2_normalization['20181030_genome_size_normalized_metagenome_cells/ml'], 
                                                                 O2_normalization['RSG_cells/ml_181030'])))

O2_normalization['20190402_cells_below_O2']= np.where (O2_normalization['RSG_cells/ml_190402']==0.000000, O2_normalization['20190402_genome_size_normalized_metagenome_cells/ml'],
                                                       (np.where(O2_normalization['RSG_cells/ml_190402']>=O2_normalization['20190402_genome_size_normalized_metagenome_cells/ml'], 0, 
                                                                 O2_normalization['20190402_genome_size_normalized_metagenome_cells/ml']-O2_normalization['RSG_cells/ml_190402'])))
O2_normalization['20190402_cells_above_O2']= np.where (O2_normalization['RSG_cells/ml_190402']==0,0,
                                                       (np.where(O2_normalization['RSG_cells/ml_190402']>=O2_normalization['20190402_genome_size_normalized_metagenome_cells/ml'], O2_normalization['20190402_genome_size_normalized_metagenome_cells/ml'], 
                                                                 O2_normalization['RSG_cells/ml_190402'])))

O2_normalization['20190709_cells_below_O2']= np.where (O2_normalization['RSG_cells/ml_190709']==0.000000, O2_normalization['20190709_genome_size_normalized_metagenome_cells/ml'],
                                                       (np.where(O2_normalization['RSG_cells/ml_190709']>=O2_normalization['20190709_genome_size_normalized_metagenome_cells/ml'], 0, 
                                                                 O2_normalization['20190709_genome_size_normalized_metagenome_cells/ml']-O2_normalization['RSG_cells/ml_190709'])))
O2_normalization['20190709_cells_above_O2']= np.where (O2_normalization['RSG_cells/ml_190709']==0,0,
                                                       (np.where(O2_normalization['RSG_cells/ml_190709']>=O2_normalization['20190709_genome_size_normalized_metagenome_cells/ml'], O2_normalization['20190709_genome_size_normalized_metagenome_cells/ml'], 
                                                                 O2_normalization['RSG_cells/ml_190709'])))

#O2_normalization=O2_normalization.fillna(0)
O2_normalization.to_csv(outdir+'O2_normalization_cells_per_ml_calculations.csv')

O2_normalization.head()

,20171102_genome_size_normalized_metagenome_cells/ml,20181030_genome_size_normalized_metagenome_cells/ml,20190402_genome_size_normalized_metagenome_cells/ml,20190709_genome_size_normalized_metagenome_cells/ml,RSG_cells/ml_171102,RSG_cells/ml_181030,RSG_cells/ml_190402,RSG_cells/ml_190709,20171102_cells_below_O2,20171102_cells_above_O2,20181030_cells_below_O2,20181030_cells_above_O2,20190402_cells_below_O2,20190402_cells_above_O2,20190709_cells_below_O2,20190709_cells_above_O2
genus,,,,,,,,,,,,,,,,
AAA164-E04,842.193966,1916.455808,369.342772,857.676906,0.0,0.000000,0.000000,0.0,842.193966,0.0,1916.455808,0.000000,369.342772,0.000000,857.676906,0.0
AAA536-G10,17472.787073,16915.541760,14403.280557,17721.566151,0.0,6829.787234,12408.759124,0.0,17472.787073,0.0,10085.754526,6829.787234,1994.521433,12408.759124,17721.566151,0.0
AG-337-I02,2227.335509,9857.221878,1075.339238,1002.756455,0.0,9106.382979,0.000000,0.0,2227.335509,0.0,750.838899,9106.382979,1075.339238,0.000000,1002.756455,0.0
AG-414-E02,125.191537,438.765267,401.157950,128.286002,0.0,0.000000,0.000000,0.0,125.191537,0.0,438.765267,0.000000,401.157950,0.000000,128.286002,0.0
AG-422-B15,1880.679178,3313.158567,13707.625915,6736.371699,0.0,0.000000,7445.255474,0.0,1880.679178,0.0,3313.158567,0.000000,6262.370440,7445.255474,6736.371699,0.0


### I have 8 GEN/timepoints where the number of cells that I detected in the RSG fraction is > than the number of cells that were calculated via the metagenome normalization. I see 2 options right now
1. just set the # of cells below detection to 0 and use the number of cells detected via RSG as my number of cells/ml for that GEN
2. Set the number of cells below detection to 0 and change the number of active cells (aka the number of cells detected via RSG to the mg normalized number of cells. I think that this is the better approach but I am not sure exactly how to change the necessary values. 

In [34]:
# load the RSG calculated O2 per gen/day file
columns=['date', 'genus', 'Avg_O2_consumed_per_cell']
RSG_avg_O2=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/O2_consumption/Average_O2_consumption_for_Genera_with_3+_RSG_SAGs_per_day.csv',
                      usecols=columns)
RSG_avg_O2

RSG_avg_O2=pd.pivot_table(RSG_avg_O2, index='genus', columns='date', values='Avg_O2_consumed_per_cell')
#RSG_avg_O2=RSG_avg_O2.reset_index()

#RSG_avg_O2['genus']=RSG_avg_O2['genus'].astype(str)
RSG_avg_O2=RSG_avg_O2.fillna(0)
#RSG_avg_O2=RSG_avg_O2.set_index('genus')

RSG_avg_O2.head()

date,171102,181030,190402,190709
genus,,,,
AAA536-G10,0.000000,0.004542,0.004337,0.00000
AG-337-I02,0.000000,0.009881,0.000000,0.00000
AG-422-B15,0.000000,0.000000,0.004214,0.00000
ASP10-02a,0.000000,0.000000,2.069933,0.00000
Amylibacter,0.478868,0.070978,0.075798,0.14474


In [35]:
#calculate the detection limit and the arbitrary value that i will be using for these cells
# in order to calculate this I found the mininum amount of O2 that was consumed by a cell at each timepoint and divided by 2



min_O2_consumption_171102=(RSG_avg_O2[RSG_avg_O2 >0][171102].min())/2
min_O2_consumption_181030=(RSG_avg_O2[RSG_avg_O2 >0][181030].min())/2
min_O2_consumption_190402=(RSG_avg_O2[RSG_avg_O2 >0][190402].min())/2
min_O2_consumption_190709=(RSG_avg_O2[RSG_avg_O2 >0][190709].min())/2

print(min_O2_consumption_171102, min_O2_consumption_181030, min_O2_consumption_190402, min_O2_consumption_190709)

0.0023300232656564266 0.0020498313547055113 0.0021006560205392797 0.0019987981496087256


In [36]:
### PROBLEM ###



In [37]:
#Use the RSG labeled O2 consumption and the RSG cells, along with the below O2 detection # of cells and the abritrary O2 value 
# to calculate an avg O2 consumption value.
# weighted_avg=((RSG_O2*above_detection_cells)+(arbitrary_O2*below_detection_cells))/mg_normalized_cells

O2_normalization['171102_above_O2_consumption']=RSG_avg_O2[171102]*O2_normalization['20171102_cells_above_O2']
O2_normalization['171102_below_O2_consumption']=min_O2_consumption_171102*O2_normalization['20171102_cells_below_O2']
O2_normalization=O2_normalization.fillna(0)
O2_normalization['171102_total_O2_consumption']=O2_normalization['171102_above_O2_consumption']+O2_normalization['171102_below_O2_consumption']

O2_normalization['181030_above_O2_consumption']=RSG_avg_O2[181030]*O2_normalization['20181030_cells_above_O2']
O2_normalization['181030_below_O2_consumption']=min_O2_consumption_181030*O2_normalization['20181030_cells_below_O2']
O2_normalization=O2_normalization.fillna(0)
O2_normalization['181030_total_O2_consumption']=O2_normalization['181030_above_O2_consumption']+O2_normalization['181030_below_O2_consumption']

O2_normalization['190402_above_O2_consumption']=RSG_avg_O2[190402]*O2_normalization['20190402_cells_above_O2']
O2_normalization['190402_below_O2_consumption']=min_O2_consumption_190402*O2_normalization['20190402_cells_below_O2']
O2_normalization=O2_normalization.fillna(0)
O2_normalization['190402_total_O2_consumption']=O2_normalization['190402_above_O2_consumption']+O2_normalization['190402_below_O2_consumption']

O2_normalization['190709_above_O2_consumption']=RSG_avg_O2[190709]*O2_normalization['20190709_cells_above_O2']
O2_normalization['190709_below_O2_consumption']=min_O2_consumption_190709*O2_normalization['20190709_cells_below_O2']
O2_normalization=O2_normalization.fillna(0)
O2_normalization['190709_total_O2_consumption']=O2_normalization['190709_above_O2_consumption']+O2_normalization['190709_below_O2_consumption']


O2_normalization['171102_weighted_avg_O2_consumption']=O2_normalization['171102_total_O2_consumption']/O2_normalization['20171102_genome_size_normalized_metagenome_cells/ml']
O2_normalization['181030_weighted_avg_O2_consumption']=O2_normalization['181030_total_O2_consumption']/O2_normalization['20181030_genome_size_normalized_metagenome_cells/ml']
O2_normalization['190402_weighted_avg_O2_consumption']=O2_normalization['190402_total_O2_consumption']/O2_normalization['20190402_genome_size_normalized_metagenome_cells/ml']
O2_normalization['190709_weighted_avg_O2_consumption']=O2_normalization['190709_total_O2_consumption']/O2_normalization['20190709_genome_size_normalized_metagenome_cells/ml']

O2_normalization.to_csv(outdir+'weighted_avg_O2_consumption_calculations.csv')
O2_normalization=O2_normalization.reset_index()
O2_normalization=O2_normalization.drop_duplicates()
O2_normalization.head()

,genus,20171102_genome_size_normalized_metagenome_cells/ml,20181030_genome_size_normalized_metagenome_cells/ml,20190402_genome_size_normalized_metagenome_cells/ml,20190709_genome_size_normalized_metagenome_cells/ml,RSG_cells/ml_171102,RSG_cells/ml_181030,RSG_cells/ml_190402,RSG_cells/ml_190709,20171102_cells_below_O2,...,190402_above_O2_consumption,190402_below_O2_consumption,190402_total_O2_consumption,190709_above_O2_consumption,190709_below_O2_consumption,190709_total_O2_consumption,171102_weighted_avg_O2_consumption,181030_weighted_avg_O2_consumption,190402_weighted_avg_O2_consumption,190709_weighted_avg_O2_consumption
0,AAA164-E04,842.193966,1916.455808,369.342772,857.676906,0.0,0.000000,0.000000,0.0,842.193966,...,0.000000,0.775862,0.775862,0.0,1.714323,1.714323,0.00233,0.002050,0.002101,0.001999
1,AAA536-G10,17472.787073,16915.541760,14403.280557,17721.566151,0.0,6829.787234,12408.759124,0.0,17472.787073,...,53.821006,4.189803,58.010809,0.0,35.421834,35.421834,0.00233,0.003056,0.004028,0.001999
2,AG-337-I02,2227.335509,9857.221878,1075.339238,1002.756455,0.0,9106.382979,0.000000,0.0,2227.335509,...,0.000000,2.258918,2.258918,0.0,2.004308,2.004308,0.00233,0.009284,0.002101,0.001999
3,AG-414-E02,125.191537,438.765267,401.157950,128.286002,0.0,0.000000,0.000000,0.0,125.191537,...,0.000000,0.842695,0.842695,0.0,0.256418,0.256418,0.00233,0.002050,0.002101,0.001999
4,AG-422-B15,1880.679178,3313.158567,13707.625915,6736.371699,0.0,0.000000,7445.255474,0.0,1880.679178,...,31.371807,13.155086,44.526893,0.0,13.464647,13.464647,0.00233,0.002050,0.003248,0.001999


In [38]:
cells2017=O2_normalization[['genus', '20171102_cells_above_O2', '20171102_cells_below_O2']].copy()
cells2017.rename(columns={'20171102_cells_above_O2':'cells_above_O2', '20171102_cells_below_O2':'cells_below_O2'}, inplace=True)
cells2017['date']=20171102
cells2018=O2_normalization[['genus', '20181030_cells_above_O2', '20181030_cells_below_O2']].copy()
cells2018.rename(columns={'20181030_cells_above_O2':'cells_above_O2', '20181030_cells_below_O2':'cells_below_O2'}, inplace=True)
cells2018['date']=20181030
cells201904=O2_normalization[['genus', '20190402_cells_above_O2', '20190402_cells_below_O2']].copy()
cells201904.rename(columns={'20190402_cells_above_O2':'cells_above_O2', '20190402_cells_below_O2':'cells_below_O2'}, inplace=True)
cells201904['date']=20190402
cells201907=O2_normalization[['genus', '20190709_cells_above_O2', '20190709_cells_below_O2']].copy()
cells201907.rename(columns={'20190709_cells_above_O2':'cells_above_O2', '20190709_cells_below_O2':'cells_below_O2'}, inplace=True)
cells201907['date']=20190709
cells201907.head()

dfs=[cells2018, cells201904, cells201907]

cells=pd.concat(dfs, ignore_index=True)
cells.to_csv(outdir+'Cells_above_and_below_O2detection_limit_per_genus.csv')
cells


,genus,cells_above_O2,cells_below_O2,date
0,AAA164-E04,0.000000,1916.455808,20181030
1,AAA536-G10,6829.787234,10085.754526,20181030
2,AG-337-I02,9106.382979,750.838899,20181030
3,AG-414-E02,0.000000,438.765267,20181030
4,AG-422-B15,0.000000,3313.158567,20181030
...,...,...,...,...
496,Verrucomicrobiales,0.000000,280.890811,20190709
497,Vibrio,0.000000,964.925624,20190709
498,Vicingaceae,0.000000,5084.185705,20190709
499,Winogradskyella,0.000000,10722.976925,20190709


In [39]:
weighted_avg_O2_per_cell=O2_normalization[['genus', '171102_weighted_avg_O2_consumption', '181030_weighted_avg_O2_consumption', 
                                          '190402_weighted_avg_O2_consumption', '190709_weighted_avg_O2_consumption']].copy()
weighted_avg_O2_per_cell.to_csv(outdir+ 'GoM_mg_abundance_weighted_average_O2_consumption_per_cell.csv')

weighted_avg_O2_per_cell.tail(10)

,genus,171102_weighted_avg_O2_consumption,181030_weighted_avg_O2_consumption,190402_weighted_avg_O2_consumption,190709_weighted_avg_O2_consumption
158,UBA952,0.002330,0.002050,0.002101,0.001999
159,UBA985,0.002330,0.002050,0.002101,0.001999
160,UBA9926,0.011640,0.002050,0.002101,0.005047
161,Ulvibacter,0.002330,0.002050,0.002101,0.001999
162,Unclassified,23.337911,0.028693,0.439139,0.012073
163,Verrucomicrobiales,0.002330,0.002050,0.002101,0.001999
164,Vibrio,0.002330,0.002050,0.002101,0.001999
165,Vicingaceae,0.002330,0.002050,0.002101,0.001999
166,Winogradskyella,0.002330,0.002050,0.002101,0.001999
167,Yoonia,0.002330,0.002050,0.002101,0.001999


I believe that I have successfully figured out how to weight the O2 consumption to account for the porportion of any given GEN that is below the detection limit of the 
RSG. Any GENs that are detected in the metagenomes but do not reach sufficient abundance in the RSG fraction (at least 3 cells) will have exactly the same avg respiration per day (1/2 the lowest detected value per day ~0.0022 fmol/cell/hr). I went also used this value to weight the average O2 consumption for the entire GEN. eg 50% of cells above detection get the avg value for those cells 0.2 and 50% are below O2 detection limit based on mg abundance so they get the lower value assigned 0.002. resulting in the average detection for this imaginary GEN to be 0.101 fmol/cell.

In [40]:
#In order to put this into the linear regression analysis I now need to melt the table and add a column with the date/column title

O2_171102=weighted_avg_O2_per_cell[['genus', '171102_weighted_avg_O2_consumption']].copy()
O2_171102['date']=20171102
O2_171102=O2_171102.rename(columns={'171102_weighted_avg_O2_consumption':'Weighted_avg_O2_consumed_per_cell'})

O2_181030=weighted_avg_O2_per_cell[['genus', '181030_weighted_avg_O2_consumption']].copy()
O2_181030['date']=20181030
O2_181030=O2_181030.rename(columns={'181030_weighted_avg_O2_consumption':'Weighted_avg_O2_consumed_per_cell'})

O2_190402=weighted_avg_O2_per_cell[['genus', '190402_weighted_avg_O2_consumption']].copy()
O2_190402['date']=20190402
O2_190402=O2_190402.rename(columns={'190402_weighted_avg_O2_consumption':'Weighted_avg_O2_consumed_per_cell'})

O2_190709=weighted_avg_O2_per_cell[['genus', '190709_weighted_avg_O2_consumption']].copy()
O2_190709['date']=20190709
O2_190709=O2_190709.rename(columns={'190709_weighted_avg_O2_consumption':'Weighted_avg_O2_consumed_per_cell'})

O2_stacked=pd.concat([O2_171102,O2_181030,O2_190402,O2_190709], ignore_index=True)
O2_stacked.to_csv(outdir+ 'Weighted_average_O2_consumption_per_cell_per_genus.csv')
O2_stacked

,genus,Weighted_avg_O2_consumed_per_cell,date
0,AAA164-E04,0.002330,20171102
1,AAA536-G10,0.002330,20171102
2,AG-337-I02,0.002330,20171102
3,AG-414-E02,0.002330,20171102
4,AG-422-B15,0.002330,20171102
...,...,...,...
663,Verrucomicrobiales,0.001999,20190709
664,Vibrio,0.001999,20190709
665,Vicingaceae,0.001999,20190709
666,Winogradskyella,0.001999,20190709


In [41]:
# now combine this file with the genus_and_konum_stcked file from notebook 05
# to create the input for the linear regression analysis 

#read file
gene_copy=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GORG_recruitment/RNA_summaries/All_GoM_RNA_seq_transcripts_per_cell_by_genus_and_KOnum_stacked.csv',
                     index_col=0)
#merge with knumber
gene_copy=gene_copy.merge(O2_stacked, on=['genus', 'date'], how='left')
gene_copy.to_csv(outdir+ 'All_GoM_RNA_seq_transcripts_per_cell_by_genus_and_function_stacked_weighted_avg_O2_consumed_ALL_genes_all_Genus.csv')

#Remove 2017 samples
gene_copy=gene_copy[gene_copy['date'] != 20171102]
gene_copy.to_csv(outdir+ 'All_GoM_RNA_seq_transcripts_per_cell_by_genus_and_function_stacked_weighted_avg_O2_consumed_ALL_genes_all_Genus_no2017.csv')

# remove genera without at least 10SAGs
with open("/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/Genera_with_10+_SAGs.txt") as file:
    tenGENs_list = [line.strip() for line in file]

gene_copy=gene_copy[gene_copy['genus'].isin(tenGENs_list)]
gene_copy.to_csv(outdir+'All_GoM_RNA_seq_transcripts_per_cell_by_genus_and_function_stacked_weighted_avg_O2_consumed_ALL_genes_10SAG_genera_no2017.csv')
gene_copy

,genus,ko,transcripts_per_cell,date,Weighted_avg_O2_consumed_per_cell
74990,Unclassified,K00003,0.000347,20181030,0.028693
74991,Unclassified,K00007,0.000000,20181030,0.028693
74992,Unclassified,K00010,0.000000,20181030,0.028693
74993,Unclassified,K00012,0.002429,20181030,0.028693
74994,Unclassified,K00013,0.000000,20181030,0.028693
...,...,...,...,...,...
254350,UBA4466,K18989,0.282012,20190709,0.001999
254351,UBA4466,K19519,0.056402,20190709,0.001999
254352,UBA4466,K21397,0.028201,20190709,0.001999
254353,UBA4466,K22491,0.028201,20190709,0.001999


In [42]:
cells_melt=pd.melt(O2_normalization, id_vars='genus', value_vars=['20171102_cells_below_O2', '20171102_cells_above_O2',
                                                                 '20181030_cells_below_O2', '20181030_cells_above_O2',
                                                                 '20190402_cells_below_O2', '20190402_cells_above_O2',
                                                                 '20190709_cells_below_O2', '20190709_cells_above_O2'],
                  value_name="cells/ml")

cells_melt['date']=cells_melt['variable'].str[:8]
cells_melt['fraction']=cells_melt['variable'].str[15:20]
cells_melt=cells_melt.drop(columns='variable')
cells_melt=cells_melt.drop_duplicates()

cells_melt

,genus,cells/ml,date,fraction
0,AAA164-E04,842.193966,20171102,below
1,AAA536-G10,17472.787073,20171102,below
2,AG-337-I02,2227.335509,20171102,below
3,AG-414-E02,125.191537,20171102,below
4,AG-422-B15,1880.679178,20171102,below
...,...,...,...,...
1331,Verrucomicrobiales,0.000000,20190709,above
1332,Vibrio,0.000000,20190709,above
1333,Vicingaceae,0.000000,20190709,above
1334,Winogradskyella,0.000000,20190709,above


In [43]:
O2_melt=pd.melt(O2_normalization, id_vars='genus', value_vars=['171102_below_O2_consumption', '171102_above_O2_consumption',
                                                               '181030_below_O2_consumption', '181030_above_O2_consumption',
                                                               '190402_below_O2_consumption', '190402_above_O2_consumption',
                                                               '190709_below_O2_consumption', '190709_above_O2_consumption'],
                value_name='fmol_O2_consumed_per_ml_per_hr')
O2_melt['date']=O2_melt['variable'].str[:6]
O2_melt['date']=O2_melt['date'].astype(int)
O2_melt['date']=O2_melt['date']+20000000
O2_melt['date']=O2_melt['date'].astype(str)
O2_melt['fraction']=O2_melt['variable'].str[7:12]
O2_melt=O2_melt.drop(columns=['variable'])
O2_melt=O2_melt.drop_duplicates()

O2_melt

,genus,fmol_O2_consumed_per_ml_per_hr,date,fraction
0,AAA164-E04,1.962332,20171102,below
1,AAA536-G10,40.712000,20171102,below
2,AG-337-I02,5.189744,20171102,below
3,AG-414-E02,0.291699,20171102,below
4,AG-422-B15,4.382026,20171102,below
...,...,...,...,...
1331,Verrucomicrobiales,0.000000,20190709,above
1332,Vibrio,0.000000,20190709,above
1333,Vicingaceae,0.000000,20190709,above
1334,Winogradskyella,0.000000,20190709,above


In [44]:
consumption=cells_melt.merge(O2_melt, how='left', on=['genus', 'date', 'fraction'])
consumption.to_csv(outdir+ 'O2_consumed_by_cells_above_and_below_detection_limit_per_genus.csv')
consumption

,genus,cells/ml,date,fraction,fmol_O2_consumed_per_ml_per_hr
0,AAA164-E04,842.193966,20171102,below,1.962332
1,AAA536-G10,17472.787073,20171102,below,40.712000
2,AG-337-I02,2227.335509,20171102,below,5.189744
3,AG-414-E02,125.191537,20171102,below,0.291699
4,AG-422-B15,1880.679178,20171102,below,4.382026
...,...,...,...,...,...
1331,Verrucomicrobiales,0.000000,20190709,above,0.000000
1332,Vibrio,0.000000,20190709,above,0.000000
1333,Vicingaceae,0.000000,20190709,above,0.000000
1334,Winogradskyella,0.000000,20190709,above,0.000000
